# Criando o _dataset_

In [1]:
import pandas as pd
from datetime import date

## Carregando dados do repositório 

In [2]:
url = 'https://raw.githubusercontent.com/adaoduque/Brasileirao_Dataset/master/campeonato-brasileiro-full.csv'

df = pd.read_csv(url)

## Renomeando colunas

In [3]:
df.columns = ['Horário', 'Dia', 'Data', 'Mandante', 'Visitante', 'Vencedor', 'Rodada',
       'Arena', 'Gols_Mandante', 'Gols_Visitante', 'Estado_Mandante',
       'Estado_Visitante', 'Estado_Vencedor']

## Criando nova coluna

In [4]:
df["Ano"] = df.Data.apply(lambda x: date.fromisoformat(x).year)

## Excluindo colunas desnecessárias

In [5]:
df.drop(labels=['Horário', 'Dia', 'Arena', 'Estado_Mandante',
       'Estado_Visitante', 'Estado_Vencedor'], axis="columns", inplace=True)

# Filtrando os dados

Queremos apenas os campeonatos a partir de 2013, pois foi quando o novo ranking da CBF surgiu.

In [6]:
df_filtrado = df.query("Ano > 2012")

In [7]:
dataset = df_filtrado[['Ano', 'Mandante', 'Visitante','Gols_Mandante', 'Gols_Visitante' ]].copy()

## Inserindo a coluna Rodada

In [8]:
dataset.insert(1, "Rodada", df_filtrado.Rodada.apply(lambda x: int(x.split(" ")[0][:-1])))

## Normalizando os nomes dos times

In [9]:
dataset[['Mandante','Visitante']] = dataset[['Mandante','Visitante']].applymap(lambda x: x.capitalize())

# Calculando o saldo do mandante

In [10]:
dataset["Saldo_Mandante"] = dataset.Gols_Mandante - dataset.Gols_Visitante

## Separando em classes

- Classe  1: vitória do mandante
- Classe  0: empate
- Classe -1: vitória do visitante

In [11]:
classe1, classe2, classe3 =  (dataset.query("Gols_Mandante {} Gols_Visitante".format(op)).index for op in (">", "==", "<")) 

In [12]:
dataset["Classe"] = 0 

In [13]:
dataset.loc[classe1, "Classe"] = 1
dataset.loc[classe3, "Classe"] = -1

## Funções para calcular saldos de gol e pontos

In [15]:
def saldo_gols(time, ano, rodada):
    main_query =  f" and Ano == {ano} and Rodada < {rodada}"
    positivo = dataset.query(f"Mandante == '{time}'" + main_query ).Saldo_Mandante
    negativo = dataset.query(f"Visitante == '{time}'" + main_query ).Saldo_Mandante
    return positivo.sum() - negativo.sum()

In [16]:
def pontuacao(time, ano, rodada):
    main_query =  f" and Ano == {ano} and Rodada < {rodada}"
    vitorias_casa = dataset.query(f"Mandante == '{time}' and Classe == 1" + main_query).Ano
    vitorias_fora = dataset.query(f"Visitante == '{time}' and Classe == -1" + main_query).Ano
    empates  = dataset.query(f"(Mandante == '{time}' or Visitante == '{time}') and Classe == 0" + main_query).Ano
    return 3*vitorias_casa.count() + 3*vitorias_fora.count() + empates.count()

# Ranking CBF

In [17]:
ranking = pd.read_csv("datasets/RankingCBF.csv")
ranking.set_index("Time", inplace=True)

In [18]:
def get_ranking(time, ano):
    ano_str = str(ano)
    return ranking.loc[time, ano_str]

# Criando e Exportanto dados

In [19]:
for ano in range(2013, 2020):
    subset = dataset.query(f"Ano == {ano} ")[["Mandante", "Visitante", "Rodada"]]
    classe = dataset.query(f"Ano == {ano} ")[["Classe"]]
    
    ranking_mandante = subset.Mandante.apply(lambda x: get_ranking(x, ano))
    pontos_mandante = subset.apply(lambda x: pontuacao(x["Mandante"], ano, x["Rodada"]), axis=1)
    saldo_mandante = subset.apply(lambda x: saldo_gols(x["Mandante"], ano, x["Rodada"]), axis=1)
    
    ranking_visitante = subset.Visitante.apply(lambda x: get_ranking(x, ano))
    pontos_visitante = subset.apply(lambda x: pontuacao(x["Visitante"], ano, x["Rodada"]), axis=1)
    saldo_visitante = subset.apply(lambda x: saldo_gols(x["Visitante"], ano, x["Rodada"]), axis=1)
    
    calculados = pd.DataFrame({
        "ranking_mandante": ranking_mandante,
        "pontos_mandante": pontos_mandante,
        "saldo_mandante": saldo_mandante,
        "ranking_visitante": ranking_visitante,
        "pontos_visitante": pontos_visitante,
        "saldo_visitante": saldo_visitante
    })
    
    salvo = pd.concat([subset, calculados, classe], axis=1)
    salvo.to_csv(f"datasets/{ano}.csv", index=False)